In [1]:
# all necessary includes

import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from nltk.corpus import stopwords

import os
import nltk
# nltk.download('stopwords')
from string import punctuation
from pymystem3 import Mystem

In [68]:
# additional functions
import re
mystem = Mystem()
garbage_words = stopwords.words('english') + stopwords.words('russian')


# will split dataframe into samples with same proportion of each document class
def SplitTrainSample(dataframe, fraction=0.7):
    '''
    1) to form arrays for each document class
    2) to split each class by np.split
    3) to shuffle rows in resulting dataframe1 and dataframe2
    '''
    types = set(dataframe['target'])
    print(types)
    dataframes = [dataframe[dataframe['target'] == doc_type] for doc_type in types]
    splited = [np.split(df.sample(frac=1), [int(fraction * len(df))]) for df in dataframes]
    test = [df[0] for df in splited]
    train = [df[1] for df in splited]
    return pd.concat(train).sample(frac=1), pd.concat(test).sample(frac=1)


def CreateVectorizer(ngram_range, max_features):
    return CountVectorizer(analyzer = "word",
                           #min_df=5,
                           #max_df=0.7,
                           tokenizer = None,
                           preprocessor = None,
                           stop_words = None,
                           ngram_range = ngram_range,
                           max_features = max_features
                         )



def TrainRandomForestClassifier(dataframe, n_estimators=300):
    ngram_range = (1, 3)
    max_features = 50000

    vectorizer = CreateVectorizer(ngram_range, max_features)
    X_train, Y_train = dataframe['X'], dataframe['target']
    
    X_train = vectorizer.fit_transform(X_train)
    #print(X_train.shape)
    #print(type(X_train))
    # X_train = X_train.toarray()

    model = RandomForestClassifier(n_estimators=n_estimators)
    model = model.fit(X_train, Y_train)
    return vectorizer, model


def CalculateAccuracy(model, vectorizer, ds, confidence_level):
    ds['predict'] = model.predict(vectorizer.transform(ds['X']))
    ds['confidence'] = np.transpose(np.amax(model.predict_proba(vectorizer.transform(ds['X'])), axis=1))
    mean_confidence = np.mean(ds['confidence']) #np.mean(model.predict_proba(vectorizer.transform(ds['X'])))
    
    precision = round(100 * len(ds[(ds['confidence'] >= confidence_level) & (ds['target'] == ds['predict'])]) / len(ds), 2)
    to_validate = round(100 * len(ds[(ds['confidence'] < confidence_level)]) / len(ds), 2)
    return precision, to_validate, mean_confidence


def BuildDataFrame(cur_folder, doc_types, file_names, doc_texts, name=''):
    files = os.listdir()
    cur_name = name[:]
    if len(cur_name) != 0:
        cur_name += '-'
    cur_name += cur_folder
    if 'text_layer' in files:
        files.remove('text_layer')
        print('in', cur_name)
        
    folders = [file for file in files if '.' not in file]
    
    for folder in folders:
        os.chdir(folder)
        BuildDataFrame(folder, doc_types, file_names, doc_texts, cur_name)
        os.chdir('..')
    if len(folders) == 0:
        for file in files:
            text_layer = ''
            if 'text_layer' in os.listdir():
                try:
                    with open('./text_layer/' + file.split('.')[0] + '.txt') as f:
                        text_layer = f.read()
                except:
                    NOT_FOUND.append((file, cur_name))
            if len(text_layer) > 0:
                doc_texts.append(text_layer)
                doc_types.append(cur_name)
                file_names.append(file)
        

In [3]:
# build dataframe
NOT_FOUND = []
doc_types, file_names, doc_texts = [], [], []

dataset_path = 'for training'
os.chdir(dataset_path)
BuildDataFrame('', doc_types, file_names, doc_texts)
os.chdir('..')
df = pd.DataFrame({'file_name': file_names, 'doc_type': doc_types, 'doc_text' : doc_texts})
# print(df)

in ad-error
in ad-invoice-first
in ad-invoice-multi
in ad-statement-first
in ad-statement-multi
in ad-upd-first
in ad-upd-multi
in ad-waybill-first
in ad-waybill-multi
in rfq-AOG
in rfq-CRITICAL
in rfq-Routine
in rfq-Spam
in sec-error
in sec-Изменение контроля
in sec-Кросс-дефолт
in sec-Обозначение прав дочерних компаний
in sec-Оговорки о коллективных действиях
in sec-Ограничение деятельности
in sec-Ограничение задолженности
in sec-Ограничение задолженности дочерних компаний
in sec-Ограничение по инвестициям
in sec-Ограничение по наслоению долговых обязательств по рангам
in sec-Ограничение по платежам
in sec-Ограничение по платежам в отношении дочерних компаний
in sec-Ограничение по предоставлению залога
in sec-Ограничение по продаже активов
in sec-Ограничение по продаже активов с обратной арендой
in sec-Ограничение по слиянию
in sec-Ограничение по транзакциям с аффилированными лицами
in sec-Случаи дефолта
in sec-Триггер рейтингов
in sec-Условие приостановки действия ковенантов
in sec-

In [4]:
# part for checking files that weren't found
ext = [x[0].split('.')[-1] for x in NOT_FOUND]
print(set(ext))
formated_not_found = ['file {} at {} not found'.format(x[0], x[1]) for x in NOT_FOUND if '' in x[0].split('.')[-1].lower()]
print(len(formated_not_found))
print('\n'.join(formated_not_found))

{'jpg', 'msg', 'JPEG', 'pdf', 'PDF'}
49
file 06014769-9ae4-458d-a7cb-8ade56355dd6.pdf at ad-error not found
file 1012b487-da5b-43b3-91b5-1862e8b07c5a.pdf at ad-error not found
file 10f6b3f1-bcaa-49c6-af90-4493d6d50cbd.pdf at ad-error not found
file 19596038-1690-4aa0-84aa-162381f70581.pdf at ad-error not found
file 1e9ba652-ca81-4eec-9208-d0dfad11469b.pdf at ad-error not found
file 1f6f9a23-d9c4-4ed8-be3b-89afd4c77568.pdf at ad-error not found
file 25cbeedc-2612-4784-86de-0e139945707b.pdf at ad-error not found
file 29dcc2e1-9a68-49c1-9d43-741cf491d5dc.pdf at ad-error not found
file 2a4d3482-25d4-403b-9a0d-4b41841d44b3.pdf at ad-error not found
file 369310e4-850d-46bb-a756-151af523810a.pdf at ad-error not found
file 41aebe14-3183-46a1-b4fd-3a1513b787c7.pdf at ad-error not found
file 4889bca2-01e1-4c31-bacb-0632f36878c8.pdf at ad-error not found
file 4a753c6d-80d7-4cc9-b0e8-0192b82f0377.PDF at ad-error not found
file 51a0488a-44cd-4956-b749-998de144527b.pdf at ad-error not found
file 544

In [5]:
def ClearText(text, garbage_words):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'[().,:-]', '', text).replace('\n', ' ')
    # text = ' '.join([word for word in text.lower().split() if word not in garbage_words and word.strip() not in punctuation])
    # tokens = mystem.lemmatize(text.lower())
    tokens = text.lower().split()
    tokens = [token for token in tokens if token not in garbage_words and token != ' ' and token.strip() not in punctuation]
    #tokens = tokens.replace('\n', '')
    text = ' '.join(tokens)
    
    return text

In [6]:
for i in range(len(df['doc_text'])):
    df['doc_text'][i] = ClearText(df['doc_text'][i], garbage_words)

df = df.rename(columns={'doc_type' : 'target', 'doc_text' : 'X'})

In [7]:
types_list = list(set(df['target']))
types_fra = [len(df[df['target']  == tp]) / len(df) for tp in types_list]
stat = pd.DataFrame({'target': types_list, 'percentage' : types_fra}).sort_values('percentage')
stat

,target,percentage
12,sec-Ограничение по инвестициям,0.000041
24,sec-Ограничение по наслоению долговых обязател...,0.000246
17,sec-Условие приостановки действия ковенантов,0.000451
1,sec-Ограничение по платежам в отношении дочерн...,0.000451
8,sec-Ограничение по платежам,0.000492
26,sec-Обозначение прав дочерних компаний,0.000534
13,sec-Ограничение задолженности дочерних компаний,0.000534
25,sec-Ограничение по продаже активов,0.000575
7,sec-Ограничение по продаже активов с обратной ...,0.000575
27,sec-Ограничение задолженности,0.000575


In [8]:
# split into training (0.6 of dataframe) validation (0.2 of dataframe) and test (0.2 of dataframe)

train, test = np.split(df.sample(frac=1), [int(0.8 * len(df))])
# train, test = SplitTrainSample(df, fraction=0.7)

In [82]:
from sklearn.model_selection import RandomizedSearchCV

n_estimators = [int(x) for x in np.linspace(start=90, stop=270, num=19)]
max_features = ['log2', 'sqrt']
max_depth = [int(x) for x in np.linspace(start=30, stop=170, num=15)] + [None]
min_samples_split = [int(x) for x in np.linspace(start=10, stop=80, num=15)]
min_samples_leaf = [int(x) for x in np.linspace(start=1, stop=10, num=5)]
bootstrap = [True, False]

param_dist = {
    'n_estimators' : n_estimators,
    #'max_features' : max_features,
    'max_depth' : max_depth,
    'min_samples_split' : min_samples_split,
    #'min_samples_leaf' : min_samples_leaf,
    #'bootstrap' : bootstrap,
    #'vctr__min_df' : [0.1, 1, 0.05, 0.2],
    #'vctr__max_df' : [1, 0.9, 0.7],
    #'vctr__max_features' : [10000,  30000, 50000]
}
#from sklearn.pipeline import Pipeline
#pipe = Pipeline([
  #  ('vctr', CountVectorizer(ngram_range=(1, 3), max_features=50000)),
  #  ('clf', RandomForestClassifier())])
print(param_dist)
#print(pipe)

{'n_estimators': [90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260, 270], 'max_depth': [30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, None], 'min_samples_split': [10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]}


In [83]:
train, test = np.split(df.sample(frac=1), [int(0.8 * len(df))])
#train, test = SplitTrainSample(df, fraction=0.7)
X_train, Y_train = train['X'], train['target']
vectorizer = CreateVectorizer((1, 3), 50000)
X_train = vectorizer.fit_transform(X_train)

In [84]:
rf = RandomForestClassifier(bootstrap=False, max_features='sqrt', min_samples_leaf=1)
rs = RandomizedSearchCV(rf,
                        param_dist,
                        n_iter=110,
                        cv=3,
                        verbose=3,
                        n_jobs=1,
                        random_state=0,
                        scoring='accuracy')
rs.fit(X_train, Y_train)

Fitting 3 folds for each of 110 candidates, totalling 330 fits


C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV 1/3] END max_depth=160, min_samples_split=75, n_estimators=160;, score=0.900 total time=  20.3s
[CV 2/3] END max_depth=160, min_samples_split=75, n_estimators=160;, score=0.896 total time=  19.8s
[CV 3/3] END max_depth=160, min_samples_split=75, n_estimators=160;, score=0.893 total time=  23.7s
[CV 1/3] END max_depth=80, min_samples_split=15, n_estimators=220;, score=0.905 total time= 1.1min
[CV 2/3] END max_depth=80, min_samples_split=15, n_estimators=220;, score=0.905 total time=  58.3s
[CV 3/3] END max_depth=80, min_samples_split=15, n_estimators=220;, score=0.895 total time=  56.6s
[CV 1/3] END max_depth=170, min_samples_split=80, n_estimators=90;, score=0.902 total time=  11.0s
[CV 2/3] END max_depth=170, min_samples_split=80, n_estimators=90;, score=0.897 total time=  11.6s
[CV 3/3] END max_depth=170, min_samples_split=80, n_estimators=90;, score=0.889 total time=  12.0s
[CV 1/3] END max_depth=40, min_samples_split=15, n_estimators=130;, score=0.892 total time=  19.4s
[CV 2/3

[CV 3/3] END max_depth=130, min_samples_split=35, n_estimators=220;, score=0.894 total time=  34.7s
[CV 1/3] END max_depth=140, min_samples_split=80, n_estimators=180;, score=0.901 total time=  21.4s
[CV 2/3] END max_depth=140, min_samples_split=80, n_estimators=180;, score=0.898 total time=  21.8s
[CV 3/3] END max_depth=140, min_samples_split=80, n_estimators=180;, score=0.892 total time=  23.2s
[CV 1/3] END max_depth=170, min_samples_split=45, n_estimators=110;, score=0.905 total time=  15.4s
[CV 2/3] END max_depth=170, min_samples_split=45, n_estimators=110;, score=0.902 total time=  15.6s
[CV 3/3] END max_depth=170, min_samples_split=45, n_estimators=110;, score=0.892 total time=  16.8s
[CV 1/3] END max_depth=170, min_samples_split=40, n_estimators=230;, score=0.905 total time=  33.1s
[CV 2/3] END max_depth=170, min_samples_split=40, n_estimators=230;, score=0.902 total time=  32.7s
[CV 3/3] END max_depth=170, min_samples_split=40, n_estimators=230;, score=0.893 total time=  35.4s


[CV 2/3] END max_depth=100, min_samples_split=25, n_estimators=240;, score=0.903 total time=  38.8s
[CV 3/3] END max_depth=100, min_samples_split=25, n_estimators=240;, score=0.894 total time=  41.0s
[CV 1/3] END max_depth=110, min_samples_split=40, n_estimators=140;, score=0.905 total time=  20.1s
[CV 2/3] END max_depth=110, min_samples_split=40, n_estimators=140;, score=0.902 total time=  19.7s
[CV 3/3] END max_depth=110, min_samples_split=40, n_estimators=140;, score=0.896 total time=  21.2s
[CV 1/3] END max_depth=110, min_samples_split=60, n_estimators=150;, score=0.905 total time=  19.1s
[CV 2/3] END max_depth=110, min_samples_split=60, n_estimators=150;, score=0.901 total time=  18.9s
[CV 3/3] END max_depth=110, min_samples_split=60, n_estimators=150;, score=0.891 total time=  20.8s
[CV 1/3] END max_depth=110, min_samples_split=30, n_estimators=220;, score=0.906 total time=  34.2s
[CV 2/3] END max_depth=110, min_samples_split=30, n_estimators=220;, score=0.901 total time=  33.7s


[CV 1/3] END max_depth=70, min_samples_split=50, n_estimators=170;, score=0.901 total time=  20.9s
[CV 2/3] END max_depth=70, min_samples_split=50, n_estimators=170;, score=0.899 total time=  21.1s
[CV 3/3] END max_depth=70, min_samples_split=50, n_estimators=170;, score=0.890 total time=  22.7s
[CV 1/3] END max_depth=80, min_samples_split=35, n_estimators=250;, score=0.904 total time=  35.6s
[CV 2/3] END max_depth=80, min_samples_split=35, n_estimators=250;, score=0.901 total time=  35.3s
[CV 3/3] END max_depth=80, min_samples_split=35, n_estimators=250;, score=0.893 total time=  38.1s
[CV 1/3] END max_depth=90, min_samples_split=30, n_estimators=230;, score=0.905 total time=  34.9s
[CV 2/3] END max_depth=90, min_samples_split=30, n_estimators=230;, score=0.900 total time=  34.6s
[CV 3/3] END max_depth=90, min_samples_split=30, n_estimators=230;, score=0.894 total time=  36.6s
[CV 1/3] END max_depth=170, min_samples_split=65, n_estimators=90;, score=0.902 total time=  11.3s
[CV 2/3] E

RandomizedSearchCV(cv=3,
                   estimator=RandomForestClassifier(bootstrap=False,
                                                    max_features='sqrt'),
                   n_iter=110, n_jobs=1,
                   param_distributions={'max_depth': [30, 40, 50, 60, 70, 80,
                                                      90, 100, 110, 120, 130,
                                                      140, 150, 160, 170,
                                                      None],
                                        'min_samples_split': [10, 15, 20, 25,
                                                              30, 35, 40, 45,
                                                              50, 55, 60, 65,
                                                              70, 75, 80],
                                        'n_estimators': [90, 100, 110, 120, 130,
                                                         140, 150, 160, 170,
                                 

In [85]:
rs.best_params_

{'n_estimators': 200, 'min_samples_split': 10, 'max_depth': 150}

In [86]:
def GetResultsDataFrame(searcher):
    searcher_df = pd.DataFrame(searcher.cv_results_).sort_values('rank_test_score').reset_index(drop=True)
    searcher_df = searcher_df.drop([
        'mean_fit_time',
        'std_fit_time',
        'std_score_time',
        'mean_score_time',
        'params',
        'split0_test_score',
        'split1_test_score',
        'split2_test_score',
        'std_test_score'
    ], axis=1)
    return searcher_df

In [87]:
rs_df = GetResultsDataFrame(rs)
rs_df.head(20)

,param_n_estimators,param_min_samples_split,param_max_depth,mean_test_score,rank_test_score
0,200,10,150,0.903965,1
1,230,15,None,0.903555,2
2,110,10,90,0.903247,3
3,90,10,140,0.903196,4
4,130,15,160,0.902837,5
5,250,15,130,0.902580,6
6,250,20,100,0.901759,7
7,140,10,80,0.901759,7
8,130,20,160,0.901708,9
9,110,20,140,0.901657,10


In [88]:
colors = ['lightgrey', 'coral', 'lightgreen', 'wheat', 'lightpink', 
          'skyblue', 'forestgreen', 'maroon', 'olivedrab']
params = ['param_' + x for x in list(param_dist.keys())]
ncols, nrows = 2, len(params) // 2 + len(params) % 2
fig, axs = plt.subplots(ncols=ncols, nrows=nrows)
sns.set(style="whitegrid", color_codes=True, font_scale=2)
fig.set_size_inches(45, 50)

for ind in range(len(params)):
    i = ind // ncols
    j = ind % ncols
    sns.barplot(x=params[ind], y='mean_test_score', data=rs_df, ax=axs[i, j], color=colors[ind])
    axs[i, j].set_ylim([.7, 1])
    axs[i, j].set_title(label=params[ind], size=30, weight='bold')

plt.show()

In [89]:
from sklearn.metrics import classification_report, accuracy_score
def ShowStatistics(model_, vectorizer_, to_show_model=True):
    if to_show_model:
        print('model:', model_)
        print('vectorizer:', vectorizer_)
    precision, to_validate, mean_confidence = CalculateAccuracy(model_, vectorizer_, test.copy(), 0.8)
    prediction = model_.predict(vectorizer_.transform(test['X']))
    print('Rows in test set: {}'.format(len(test)))
    print('Statistics:')
    print('Precision = {}%; Ratio to validate = {}%; at confidence level = {}'.format(precision, to_validate, 0.8))
    print('Mean accuracy: {}%'.format(round(100 * accuracy_score(test['target'], prediction), 2)))
    print('Mean confidence: {}%'.format(round(100 * mean_confidence, 2)))
    print(classification_report(test['target'], prediction))

In [90]:
rs.best_estimator_

RandomForestClassifier(bootstrap=False, max_depth=150, max_features='sqrt',
                       min_samples_split=10, n_estimators=200)

In [91]:
ShowStatistics(rs.best_estimator_, vectorizer)

model: RandomForestClassifier(bootstrap=False, max_depth=150, max_features='sqrt',
                       min_samples_split=10, n_estimators=200)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 76.86%; Ratio to validate = 20.78%; at confidence level = 0.8
Mean accuracy: 90.07%
Mean confidence: 88.61%
                                                           precision    recall  f1-score   support

                                         ad-invoice-first       0.94      0.94      0.94       834
                                         ad-invoice-multi       0.89      0.72      0.80        92
                                       ad-statement-first       0.98      1.00      0.99      1117
                                       ad-statement-multi       0.98      0.89      0.93       155
                                             ad-upd-first       0.92      0.94      0.93       732
                                    

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [92]:
def GetBestArgs(rs_df, param):
    a = rs_df.groupby(param).mean_test_score.mean().to_dict()
    srtd = [(a[key], key) for key in a.keys()]
    srtd.sort(key=lambda x: -x[0])
    srtd = [x[1] for x in srtd]
    return srtd

In [93]:
from sklearn.model_selection import GridSearchCV
best_n_estimators = GetBestArgs(rs_df, 'param_n_estimators')[:5]
#best_max_features = GetBestArgs(rs_df, 'param_max_features')[:1]
best_max_depth = [None] + GetBestArgs(rs_df, 'param_max_depth')[:4]
best_min_samples_split = GetBestArgs(rs_df, 'param_min_samples_split')[:4]
#best_min_samples_leaf = GetBestArgs(rs_df, 'param_min_samples_leaf')[:3]
#best_bootstrap = [False, True]

#best_min_df = GetBestArgs(rs_df, 'param_vctr__min_df')[:2]
#best_max_df = GetBestArgs(rs_df, 'param_vctr__max_df')[:2]
param_grid = {
    'n_estimators' : best_n_estimators,
    #'max_features' : best_max_features,
    'max_depth' : best_max_depth,
    'min_samples_split' : best_min_samples_split,
    #'min_samples_leaf' : best_min_samples_leaf,
    #'bootstrap' : best_bootstrap,
    #'vctr__min_df' : best_min_df,
    #'vctr__max_df' : best_max_df
}

#pipe = Pipeline([
   # ('vctr', CountVectorizer(ngram_range=(1, 3), max_features=50000)),
   # ('clf', RandomForestClassifier())])
print(param_grid)
#print(pipe)

{'n_estimators': [220, 200, 230, 120, 90], 'max_depth': [None, 100, 160, 130, 80], 'min_samples_split': [10, 35, 15, 30]}


In [94]:
train, test = np.split(df.sample(frac=1), [int(0.8 * len(df))])
#train, test = SplitTrainSample(df, fraction=0.7)
X_train, Y_train = train['X'], train['target']
vectorizer = CreateVectorizer((1, 3), 50000)
X_train = vectorizer.fit_transform(X_train)

In [95]:
rf = RandomForestClassifier()
gs = GridSearchCV(rf, param_grid, cv=3, verbose=3, n_jobs=1)
gs.fit(X_train, Y_train)
print(gs.best_estimator_)
print(gs.best_params_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\model_selection\_split.py:680: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  UserWarning,


[CV 1/3] END max_depth=None, min_samples_split=10, n_estimators=220;, score=0.898 total time= 1.2min
[CV 2/3] END max_depth=None, min_samples_split=10, n_estimators=220;, score=0.897 total time= 1.2min
[CV 3/3] END max_depth=None, min_samples_split=10, n_estimators=220;, score=0.897 total time= 1.3min
[CV 1/3] END max_depth=None, min_samples_split=10, n_estimators=200;, score=0.899 total time= 1.1min
[CV 2/3] END max_depth=None, min_samples_split=10, n_estimators=200;, score=0.896 total time= 1.0min
[CV 3/3] END max_depth=None, min_samples_split=10, n_estimators=200;, score=0.897 total time= 1.1min
[CV 1/3] END max_depth=None, min_samples_split=10, n_estimators=230;, score=0.896 total time= 1.3min
[CV 2/3] END max_depth=None, min_samples_split=10, n_estimators=230;, score=0.895 total time= 1.2min
[CV 3/3] END max_depth=None, min_samples_split=10, n_estimators=230;, score=0.896 total time= 1.3min
[CV 1/3] END max_depth=None, min_samples_split=10, n_estimators=120;, score=0.896 total tim

[CV 2/3] END max_depth=100, min_samples_split=35, n_estimators=230;, score=0.893 total time=  40.2s
[CV 3/3] END max_depth=100, min_samples_split=35, n_estimators=230;, score=0.893 total time=  32.8s
[CV 1/3] END max_depth=100, min_samples_split=35, n_estimators=120;, score=0.895 total time=  36.0s
[CV 2/3] END max_depth=100, min_samples_split=35, n_estimators=120;, score=0.892 total time=  49.9s
[CV 3/3] END max_depth=100, min_samples_split=35, n_estimators=120;, score=0.891 total time=  52.4s
[CV 1/3] END max_depth=100, min_samples_split=35, n_estimators=90;, score=0.894 total time=  39.1s
[CV 2/3] END max_depth=100, min_samples_split=35, n_estimators=90;, score=0.893 total time=  38.5s
[CV 3/3] END max_depth=100, min_samples_split=35, n_estimators=90;, score=0.893 total time=  28.0s
[CV 1/3] END max_depth=100, min_samples_split=15, n_estimators=220;, score=0.898 total time=  42.5s
[CV 2/3] END max_depth=100, min_samples_split=15, n_estimators=220;, score=0.894 total time= 1.4min
[CV

[CV 1/3] END max_depth=160, min_samples_split=30, n_estimators=220;, score=0.896 total time= 1.7min
[CV 2/3] END max_depth=160, min_samples_split=30, n_estimators=220;, score=0.895 total time= 1.5min
[CV 3/3] END max_depth=160, min_samples_split=30, n_estimators=220;, score=0.894 total time=  34.3s
[CV 1/3] END max_depth=160, min_samples_split=30, n_estimators=200;, score=0.897 total time=  34.1s
[CV 2/3] END max_depth=160, min_samples_split=30, n_estimators=200;, score=0.893 total time= 1.5min
[CV 3/3] END max_depth=160, min_samples_split=30, n_estimators=200;, score=0.895 total time= 1.6min
[CV 1/3] END max_depth=160, min_samples_split=30, n_estimators=230;, score=0.896 total time= 1.0min
[CV 2/3] END max_depth=160, min_samples_split=30, n_estimators=230;, score=0.892 total time=  34.0s
[CV 3/3] END max_depth=160, min_samples_split=30, n_estimators=230;, score=0.895 total time= 1.5min
[CV 1/3] END max_depth=160, min_samples_split=30, n_estimators=120;, score=0.895 total time=  55.2s


[CV 3/3] END max_depth=80, min_samples_split=10, n_estimators=230;, score=0.893 total time= 2.7min
[CV 1/3] END max_depth=80, min_samples_split=10, n_estimators=120;, score=0.895 total time=  38.2s
[CV 2/3] END max_depth=80, min_samples_split=10, n_estimators=120;, score=0.895 total time= 1.7min
[CV 3/3] END max_depth=80, min_samples_split=10, n_estimators=120;, score=0.896 total time= 1.9min
[CV 1/3] END max_depth=80, min_samples_split=10, n_estimators=90;, score=0.896 total time=  39.3s
[CV 2/3] END max_depth=80, min_samples_split=10, n_estimators=90;, score=0.895 total time=  25.1s
[CV 3/3] END max_depth=80, min_samples_split=10, n_estimators=90;, score=0.894 total time=  43.9s
[CV 1/3] END max_depth=80, min_samples_split=35, n_estimators=220;, score=0.894 total time= 1.5min
[CV 2/3] END max_depth=80, min_samples_split=35, n_estimators=220;, score=0.892 total time= 1.4min
[CV 3/3] END max_depth=80, min_samples_split=35, n_estimators=220;, score=0.891 total time=  50.3s
[CV 1/3] END 

In [96]:
gs_df = GetResultsDataFrame(gs)
gs_df.head(20)

,param_max_depth,param_min_samples_split,param_n_estimators,mean_test_score,rank_test_score
0,130,10,120,0.897963,1
1,130,10,90,0.897604,2
2,None,10,220,0.897502,3
3,None,10,200,0.897450,4
4,100,10,200,0.897296,5
5,100,10,90,0.897245,6
6,130,10,220,0.897194,7
7,160,10,200,0.897194,8
8,None,15,90,0.897194,9
9,160,10,230,0.897143,10


In [97]:
ShowStatistics(gs.best_estimator_, vectorizer)

model: RandomForestClassifier(max_depth=130, min_samples_split=10, n_estimators=120)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 75.48%; Ratio to validate = 22.38%; at confidence level = 0.8
Mean accuracy: 89.95%
Mean confidence: 87.63%


C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


                                                              precision    recall  f1-score   support

                                            ad-invoice-first       0.94      0.93      0.94       828
                                            ad-invoice-multi       0.98      0.67      0.79       127
                                          ad-statement-first       0.98      0.99      0.99      1148
                                          ad-statement-multi       0.98      0.92      0.95       160
                                                ad-upd-first       0.90      0.95      0.92       714
                                                ad-upd-multi       0.85      0.83      0.84       149
                                            ad-waybill-first       0.94      0.97      0.95       293
                                            ad-waybill-multi       0.96      0.68      0.79        34
                                                     rfq-AOG       0.82      0.57

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
ShowStatistics(gs.best_estimator_, vectorizer)

model: RandomForestClassifier(min_samples_split=25, n_estimators=260)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 73.94%; Ratio to validate = 24.29%; at confidence level = 0.8
Mean accuracy: 90.19%
                                                              precision    recall  f1-score   support

                                            ad-invoice-first       0.94      0.92      0.93       812
                                            ad-invoice-multi       0.96      0.73      0.83       103
                                          ad-statement-first       0.97      1.00      0.99      1138
                                          ad-statement-multi       0.99      0.88      0.93       144
                                                ad-upd-first       0.90      0.96      0.93       712
                                                ad-upd-multi       0.91      0.82      0.86       158
                

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [56]:
# write model to file
if 'models' not in os.listdir():
    os.mkdir('models')
with open('./models/doc_classifyer-model', 'wb') as f:
    pickle.dump(model, f)
with open('./models/doc_classifyer-vectorizer', 'wb') as f:
    pickle.dump(vectorizer, f)

In [32]:
# read model from file
with open('./models/doc_classifyer-model', 'rb') as f:
    model = pickle.load(f)
with open('./models/doc_classifyer-vectorizer', 'rb') as f:
    vectorizer = pickle.load(f)

In [33]:
ShowStatistics(model, vectorizer)

model: RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_split=20, n_estimators=127)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 68.53%; Ratio to validate = 31.1%; at confidence level = 0.8
Mean accuracy: 91.88%
                                                              precision    recall  f1-score   support

                                            ad-invoice-first       0.97      0.95      0.96       837
                                            ad-invoice-multi       0.96      0.73      0.83       108
                                          ad-statement-first       0.99      1.00      0.99      1144
                                          ad-statement-multi       0.99      0.93      0.96       150
                                                ad-upd-first       0.93      0.98      0.96       711
                                                ad-upd-mul

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [22]:
ShowStatistics(model, vectorizer)

model: RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_split=20, n_estimators=127)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 67.77%; Ratio to validate = 31.64%; at confidence level = 0.8
Mean accuracy: 92.53%
                                                              precision    recall  f1-score   support

                                            ad-invoice-first       0.98      0.94      0.96       812
                                            ad-invoice-multi       0.94      0.80      0.86        99
                                          ad-statement-first       0.99      1.00      0.99      1157
                                          ad-statement-multi       0.99      0.95      0.97       164
                                                ad-upd-first       0.92      0.99      0.95       719
                                                ad-upd-mu

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [31]:
ShowStatistics(model, vectorizer)

model: RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_split=20, n_estimators=127)
vectorizer: CountVectorizer(max_features=50000, ngram_range=(1, 3))
Rows in test set: 4874
Statistics:
Precision = 68.86%; Ratio to validate = 30.84%; at confidence level = 0.8
Mean accuracy: 91.61%
                                                           precision    recall  f1-score   support

                                         ad-invoice-first       0.98      0.93      0.96       831
                                         ad-invoice-multi       0.96      0.77      0.85        94
                                       ad-statement-first       0.99      1.00      0.99      1154
                                       ad-statement-multi       0.98      0.93      0.96       132
                                             ad-upd-first       0.89      0.99      0.94       712
                                             ad-upd-multi       0.96      0

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [58]:
ShowStatistics(model, vectorizer)

model: RandomForestClassifier(bootstrap=False, max_features='sqrt',
                       min_samples_split=20, n_estimators=127)
Rows in test set: 4874
Statistics:
Precision = 68.2%; Ratio to validate = 31.43%; at confidence level = 0.8
Mean accuracy: 91.77%
                                                           precision    recall  f1-score   support

                                         ad-invoice-first       0.97      0.94      0.96       865
                                         ad-invoice-multi       0.94      0.70      0.80        94
                                       ad-statement-first       0.99      1.00      0.99      1119
                                       ad-statement-multi       0.99      0.95      0.97       147
                                             ad-upd-first       0.92      0.99      0.96       690
                                             ad-upd-multi       0.89      0.89      0.89       148
                                         ad-w

C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\otrm\AppData\Roaming\Python\Python37\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
